# Project "Revenu des adults" (Adult income)

## <a name="setup"></a> Configuration générale

In [41]:
import os

print ('Projet : %s' %os.getcwd())

workingDir = '/Volumes/Data/data_analysis/EILV/'
os.chdir(workingDir)

projectDir = '%s/projects/L2M-alimentation/' %os.getcwd()


#from utilities.ReportMagics import *

#%reset_all

Projet : /Volumes/Data/data_analysis/EILV


In [42]:
#%%add_conf_code_to_report
import os
# workingDir = '.'
# os.chdir(workingDir)

conf_file = '%sconfig-L2M-alimentation.yaml' %projectDir
Sample_Size = 10000

export_dir = '%stmp/' %projectDir
print ('Fichier de configuration: %s' %conf_file)

Fichier de configuration: /Volumes/Data/data_analysis/EILV/projects/L2M-alimentation/config-L2M-alimentation.yaml


### Importer les paquets nécessaires et configurer les paramètres de l'environnement

In [43]:
#%%add_conf_code_to_report

import pandas as pd
import numpy as np
import os
#os.chdir(workingDir)
import collections
import matplotlib
import io
import sys
import operator

import nbformat as nbf
from IPython.core.display import HTML
from IPython.display import display
from ipywidgets import interact, interactive,fixed
from IPython.display import Javascript, display,HTML
from ipywidgets import widgets, VBox
import ipywidgets
import IPython
from IPython.display import clear_output
import scipy.stats as stats
from statsmodels.graphics.mosaicplot import mosaic
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os
import errno
import seaborn as sns
from string import Template
from functools import partial
from collections import OrderedDict

# Utility Classes
from utilities.ConfUtility import * 
from utilities.ReportGeneration import *
from utilities.UniVarAnalytics import *
from utilities.MultiVarAnalytics import *

%matplotlib inline

# Plotly
import plotly.offline as py
from plotly import tools as tls
py.init_notebook_mode(connected=True) #initiate the Plotly Notebook mode

#DEBUG=0

font={'family':'normal','weight':'normal','size':8}
matplotlib.rc('font',**font)
matplotlib.rcParams['figure.figsize'] = (12.0, 5.0)
matplotlib.rc('xtick', labelsize=9) 
matplotlib.rc('ytick', labelsize=9)
matplotlib.rc('axes', labelsize=10)
matplotlib.rc('axes', titlesize=10)
sns.set_style('whitegrid')

print ('Importation finie !')

Importation finie !


## <a name="read and summarize"></a> Lecture récapitulative des données

### Lire les données et déduire les types de colonnes

In [46]:
#%%add_conf_code_to_report

conf_dict = ConfUtility.parse_yaml(conf_file)

# Read in data from local file or SQL server
if 'DataSource' not in conf_dict:
    df=pd.read_csv('%s%s' %(projectDir,conf_dict['DataFilePath'][0]), skipinitialspace=True, encoding='utf-16')
else:
    import pyodbc
    cnxn = pyodbc.connect('driver=MySQL ODBC 3.51 Driver;server={};database={};Uid={};Pwd={}'.format(
            conf_dict['Server'], conf_dict['Database'],conf_dict['Username'],conf_dict['Password']))
    df = pd.read_sql(conf_dict['Query'],cnxn)

# Making sure that we are not reading any extra column
df = df[[each for each in df.columns if 'Unnamed' not in each]]

# Sampling Data if data size is larger than 10k
df0 = df # df0 is the unsampled data. Will be used in data exploration and analysis where sampling is not needed
         # However, keep in mind that your final report will always be based on the sampled data. 
if Sample_Size < df.shape[0]:
    df = df.sample(Sample_Size)

# change float data types
if 'FloatDataTypes' in conf_dict:   
    for col_name in conf_dict['FloatDataTypes']:
        df[col_name] = df[col_name].astype(float)      
        
# Getting the list of categorical columns if it was not there in the yaml file
if 'CategoricalColumns' not in conf_dict:
    conf_dict['CategoricalColumns'] = list(set(list(df.select_dtypes(exclude=[np.number]).columns)))

# Getting the list of numerical columns if it was not there in the yaml file
if 'NumericalColumns' not in conf_dict:
    conf_dict['NumericalColumns'] = list(df.select_dtypes(include=[np.number]).columns)    

# Exclude columns that we do not need
if 'ColumnsToExclude' in conf_dict:
    conf_dict['CategoricalColumns'] = list(set(conf_dict['CategoricalColumns'])-set(conf_dict['ColumnsToExclude']))
    conf_dict['NumericalColumns'] = list(set(conf_dict['NumericalColumns'])-set(conf_dict['ColumnsToExclude']))

# Ordering the categorical variables according to the number of unique categories
filtered_cat_columns = []
temp_dict = {}
for cat_var in conf_dict['CategoricalColumns']:
    temp_dict[cat_var] = len(np.unique(df[cat_var]))
sorted_x = sorted(temp_dict.items(), key=operator.itemgetter(0), reverse=True)
conf_dict['CategoricalColumns'] = [x for (x,y) in sorted_x]

ConfUtility.dict_to_htmllist(conf_dict,['Target','CategoricalColumns','NumericalColumns'])

### Lister les n-premières lignes (par défault, n=5)

In [47]:
def custom_head(df,NoOfRows):
    return HTML(df.head(NoOfRows).style.set_table_attributes("class='table'").render())
i = interact(custom_head,df=fixed(df0), NoOfRows=ipywidgets.IntSlider(min=0, max=1000, step=1, \
                                                                     value=5, description='Nombre de ligne'))

interactive(children=(IntSlider(value=5, description='Nombre de ligne', max=1000), Output()), _dom_classes=('w…

### Afficher les dimensions des données (lignes, colonnes)

In [48]:
print ('{} lignes | {} colonnes'.format(df0.shape[0],df0.shape[1]))

1865 lignes | 52 colonnes


### Imprimer les noms des variables (colonnes)

In [49]:
col_names = ','.join(each for each in list(df.columns))
print("Les variables sont :" + col_names)

Les variables sont :id_famille,date_deb,A1_A_cereales,A1_B_racines,A1_C_noix,A1_D_legumes,A1_E_viandes,A1_F_fruits,A1_G_par_vendeurs,A1_H_laitiers,A1_I_graisses,A1_J_sucres,A1_K_epices,A1_L_boissons,B2_a_moins_apprecies,B2_b_portions_limites,B2_c_reduire_nbr_repas,B2_d_restreindre_adultes,B2_e_emprunts,B3_a_repas_adultes,B3_b_repas_enfants,QB5_Avril_manque,QB5_Mai_manque,QB5_Juin_manque,QB5_Juillet_manque,B6_9_nom_autres_causes,QB7_1_reduire_repas,QB7_2_limiter_portions,QB7_3_alim_moins_apprecies,QB7_4_changer_prep,QB7_5_emprunt_argent,QB7_6_reporter_articles,QB7_7_reporter_education,QB7_8_vendre_propriete,date_fin,grappe,B1_crainte_sous_alim_7j,B4_manque_alim_4m,B6_cause,som,B8_mange_moins,REGION,taille_famille,MILIEU,POIDS,categorie_consommation,taille_famille_gp,genre_chef_f,age_chef_f,age_mean,age_groupe,niveau_instruction_chef_f


### Afficher les types de colonnes

In [50]:
print("Types des variables:")
df.dtypes

Types des variables:


id_famille                      int64
date_deb                       object
A1_A_cereales                   int64
A1_B_racines                    int64
A1_C_noix                       int64
A1_D_legumes                    int64
A1_E_viandes                    int64
A1_F_fruits                     int64
A1_G_par_vendeurs               int64
A1_H_laitiers                   int64
A1_I_graisses                   int64
A1_J_sucres                     int64
A1_K_epices                     int64
A1_L_boissons                   int64
B2_a_moins_apprecies            int64
B2_b_portions_limites           int64
B2_c_reduire_nbr_repas          int64
B2_d_restreindre_adultes        int64
B2_e_emprunts                   int64
B3_a_repas_adultes            float64
B3_b_repas_enfants              int64
QB5_Avril_manque               object
QB5_Mai_manque                 object
QB5_Juin_manque                object
QB5_Juillet_manque             object
B6_9_nom_autres_causes         object
QB7_1_reduir

## <a name="individual variable"></a>Extraire les statistiques descriptives de chaque colonne

In [51]:
def num_missing(x):
    return len(x.index)-x.count()

def num_unique(x):
    return len(np.unique(x))

temp_df = df0.describe().T
missing_df = pd.DataFrame(df0.apply(num_missing, axis=0)) 
missing_df.columns = ['données manquantes']
unq_df = pd.DataFrame(df0.apply(num_unique, axis=0))
unq_df.columns = ['unique']
types_df = pd.DataFrame(df0.dtypes)
types_df.columns = ['Type de données']

### Afficher les statistiques descriptives des variables quantitatives

In [52]:
summary_df = temp_df.join(missing_df).join(unq_df).join(types_df)
summary_df

,count,mean,std,min,25%,50%,75%,max,données manquantes,unique,Type de données
id_famille,1865.0,27148.705094,19539.094785,101.0000,8725.000000,25421.000000,43030.00000,61430.0,0,1865,int64
A1_A_cereales,1865.0,6.810188,0.954098,0.0000,7.000000,7.000000,7.00000,7.0,0,8,int64
A1_B_racines,1865.0,2.764075,2.359825,0.0000,1.000000,2.000000,4.00000,7.0,0,8,int64
A1_C_noix,1865.0,1.877212,1.480720,0.0000,1.000000,2.000000,3.00000,7.0,0,8,int64
A1_D_legumes,1865.0,5.207507,2.135523,0.0000,4.000000,6.000000,7.00000,7.0,0,8,int64
A1_E_viandes,1865.0,2.694906,2.114405,0.0000,1.000000,2.000000,4.00000,7.0,0,8,int64
A1_F_fruits,1865.0,3.068097,2.810766,0.0000,0.000000,2.000000,7.00000,7.0,0,8,int64
A1_G_par_vendeurs,1865.0,0.356032,1.172282,0.0000,0.000000,0.000000,0.00000,7.0,0,8,int64
A1_H_laitiers,1865.0,1.899732,2.564223,0.0000,0.000000,0.000000,3.00000,7.0,0,8,int64
A1_I_graisses,1865.0,5.671850,2.092185,0.0000,5.000000,7.000000,7.00000,7.0,0,8,int64


### Afficher les statistiques descriptives des variables qualitatives

In [53]:
col_names = list(types_df.index) #Get all col names
num_cols = len(col_names)
index = range(num_cols)
cat_index = []
for i in index: #Find the indices of columns in Categorical columns
    if col_names[i] in conf_dict['CategoricalColumns']:
        cat_index.append(i)
summary_df_cat = missing_df.join(unq_df).join(types_df.iloc[cat_index], how='inner') #Only summarize categorical columns
summary_df_cat

,données manquantes,unique,Type de données
QB5_Avril_manque,0,3,object
QB5_Mai_manque,0,3,object
QB5_Juin_manque,0,3,object
QB5_Juillet_manque,0,3,object
B6_9_nom_autres_causes,0,192,object
QB7_1_reduire_repas,0,3,object
QB7_2_limiter_portions,0,3,object
QB7_3_alim_moins_apprecies,0,3,object
QB7_4_changer_prep,0,3,object
QB7_5_emprunt_argent,0,3,object


## <a name="individual variables"></a>Explorer des variables individuelles

### Explorer la variable cible

In [54]:
w1_value,w2_value,w3_value = '','',''
w1, w2, w3, w4 = None, None, None, None
w1 = widgets.Dropdown(
        options=[conf_dict['Target']],
        value=conf_dict['Target'],
        description='Variable cible:',
    )

if conf_dict['Target'] in conf_dict['CategoricalColumns']: 
    i = interactive(TargetAnalytics.custom_barplot, df=fixed(df), col1=w1)
else:
    i = interactive(NumericAnalytics.custom_barplot, df=fixed(df), col1=w1)
display(i)

interactive(children=(Dropdown(description='Variable cible:', options=('B4_manque_alim_4m',), value='B4_manque…

### Explorer des variables numériques individuelles et tester la normalité (sur des données échantillonnées)

In [55]:
w1_value, w2_value, w3_value = '', '', ''
w1, w2, w3, w4 = None, None, None, None
w1 = widgets.Dropdown(
        options=conf_dict['NumericalColumns'],
        value=conf_dict['NumericalColumns'][0],
        description='Numeric Variable:',
    )

i = interactive(NumericAnalytics.custom_barplot, df=fixed(df), col1=w1)
display(i)

interactive(children=(Dropdown(description='Numeric Variable:', options=('A1_A_cereales', 'A1_B_racines', 'A1_…

### Explorer des variables qualitatives individuelles (triées par fréquences)

In [56]:
w1_value, w2_value, w3_value = '', '', ''
w1, w2, w3, w4 = None, None, None, None
w1 = widgets.Dropdown(
    options = conf_dict['CategoricalColumns'],
    value = conf_dict['CategoricalColumns'][0],
    description = 'Categorical Variable:',
)
# w1.observe (handle_change,'value')
i = interactive(CategoricAnalytics.custom_barplot, df=fixed(df), col1=w1)

display(i)

interactive(children=(Dropdown(description='Categorical Variable:', options=('taille_famille_gp', 'niveau_inst…

## <a name="multiple variables"></a>Explorer les interactions entre les variables

### <a name="rank variables"></a>Classer les variables en fonction des relations linéaires avec la variable de référence (sur les données échantillonnées)

In [57]:
cols_list = [conf_dict['Target']] + conf_dict['NumericalColumns'] + conf_dict['CategoricalColumns'] #Make target the default reference variable
cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
w1 = widgets.Dropdown(    
    options=cols_list,
    value=cols_list[0],
    description='Ref Var:'
)
w2 = ipywidgets.Text(value="5", description='Top Num Vars:')
w3 = ipywidgets.Text(value="5", description='Top Cat Vars:')
i = interactive(InteractionAnalytics.rank_associations, df=fixed(df), conf_dict=fixed(conf_dict), col1=w1, col2=w2, col3=w3)
display(i)

interactive(children=(Dropdown(description='Ref Var:', options=('B4_manque_alim_4m', 'A1_A_cereales', 'A1_B_ra…

### <a name="two categorical"></a>Explorer les interactions entre les variables qualitatives

In [59]:
w1, w2, w3, w4 = None, None, None, None

if conf_dict['Target'] in conf_dict['CategoricalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['CategoricalColumns']
    
w1 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Var Qual 1:'
)
w2 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[1],
    description='Var Qual 2:'
)
i = interactive(InteractionAnalytics.categorical_relations, df=fixed(df), col1=w1, col2=w2)
display(i)

interactive(children=(Dropdown(description='Var Qual 1:', options=('B4_manque_alim_4m', 'taille_famille_gp', '…

### <a name="two numerical"></a>Explorer les interactions entre les variables quantitatives (sur les données échantillonnées)

In [60]:
w1, w2, w3, w4 = None, None, None, None

if conf_dict['Target'] in conf_dict['NumericalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['NumericalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['NumericalColumns']
w1 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Var quanti 1:'
)
w2 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[1],
    description='Var quanti 2:'
)
i = interactive(InteractionAnalytics.numerical_relations, df=fixed(df), col1=w1, col2=w2)
display(i)

interactive(children=(Dropdown(description='Var quanti 1:', options=('A1_A_cereales', 'A1_B_racines', 'A1_C_no…

### Explorer la matrice de corrélation entre les variables quantitatives

In [61]:
w1, w2, w3, w4 = None, None, None, None
w1 = widgets.Dropdown(
    options=['pearson','kendall','spearman'],
    value='pearson',
    description='Methode:'
)
i = interactive(InteractionAnalytics.numerical_correlation, df=fixed(df), conf_dict=fixed(conf_dict),\
                                         col1=w1)

display(i)

interactive(children=(Dropdown(description='Methode:', options=('pearson', 'kendall', 'spearman'), value='pear…

### <a name="numerical and categorical"></a>Explorer les interactions entre les variables quantitative et qualitative

In [62]:
w1, w2, w3, w4 = None, None, None, None

if conf_dict['Target'] in conf_dict['NumericalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['NumericalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['NumericalColumns']
    
w1 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Quantitative:'
)

if conf_dict['Target'] in conf_dict['CategoricalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['CategoricalColumns']
    
w2 = widgets.Dropdown(
    options=cols_list,
    value=cols_list[0],
    description='Qualitative:'
)
i = interactive(InteractionAnalytics.nc_relation, df=fixed(df), \
                                                conf_dict=fixed(conf_dict), col1=w1, col2=w2, \
                                                col3=fixed(w3))

display(i)

interactive(children=(Dropdown(description='Quantitative:', options=('A1_A_cereales', 'A1_B_racines', 'A1_C_no…

### <a name="two numerical and categorical"></a>Explorer les interactions entre deux variables quantitatives et une variable qualitatives (sur des données échantillonnées)

In [63]:
w1, w2, w3, w4 = None, None, None, None

if conf_dict['Target'] in conf_dict['NumericalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['NumericalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['NumericalColumns']
    
w1 = widgets.Dropdown(
    options = cols_list,
    value = cols_list[0],
    description = 'Var Qttv 1:'
)
w2 = widgets.Dropdown(
    options = cols_list,
    value = cols_list[1],
    description = 'Var Qttv 2:'
)

if conf_dict['Target'] in conf_dict['CategoricalColumns']:
    cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
    cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
else:
    cols_list = conf_dict['CategoricalColumns']
    
w3 = widgets.Dropdown(
    options = cols_list,
    value = cols_list[0],
    description = 'Legend Cat Var:'
)
i = interactive(InteractionAnalytics.nnc_relation, df=fixed(df),conf_dict=fixed(conf_dict), col1=w1, col2=w2, col3=w3)
display(i)

interactive(children=(Dropdown(description='Var Qttv 1:', options=('A1_A_cereales', 'A1_B_racines', 'A1_C_noix…

## <a name="pca"></a>Visualiser les données quantitatives par analyse en composantes principales (sur les données échantillonnées)

### Projection 2D par ACP

In [64]:
num_numeric = len(conf_dict['NumericalColumns'])
if  num_numeric > 3:
    w1, w2, w3, w4, w5 = None, None, None, None, None
    if conf_dict['Target'] in conf_dict['CategoricalColumns']:
        cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
        cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
    else:
        cols_list = conf_dict['CategoricalColumns']
    w1 = widgets.Dropdown(
        options = cols_list,
        value = cols_list[0],
        description = 'Legend:',
        width = 10
    )
    w2 = widgets.Dropdown(
        options = [str(x) for x in np.arange(1,num_numeric+1)],
        value = '1',
        width = 1,
        description='PC at X-Axis:'
    )
    w3 = widgets.Dropdown(
        options = [str(x) for x in np.arange(1,num_numeric+1)],
        value = '2',
        description = 'PC at Y-Axis:'
    )
    i = interactive(InteractionAnalytics.numerical_pca, df=fixed(df), conf_dict=fixed(conf_dict), col1=w1, col2=w2, col3=w3)

    
    display(i)

interactive(children=(Dropdown(description='Legend:', options=('B4_manque_alim_4m', 'taille_famille_gp', 'nive…

### Projection 3D par ACP

In [65]:
if len(conf_dict['NumericalColumns']) > 3:
    if conf_dict['Target'] in conf_dict['CategoricalColumns']:
        cols_list = [conf_dict['Target']] + conf_dict['CategoricalColumns'] #Make target the default reference variable
        cols_list = list(OrderedDict.fromkeys(cols_list)) #remove variables that might be duplicates with target
    else:
        cols_list = conf_dict['CategoricalColumns']
    w1, w2, w3, w4 = None, None, None, None
    w1 = widgets.Dropdown(
        options=cols_list,
        value=cols_list[0],
        description='Legend:'
    )
    w2 = ipywidgets.IntSlider(min=-180, max=180, step=5, value=30, description='Angle')
    i = interactive(InteractionAnalytics.pca_3d, df=fixed(df), conf_dict=fixed(conf_dict), col1=w1, col2=w2, col3=fixed(w3))
    display(i)

interactive(children=(Dropdown(description='Legend:', options=('B4_manque_alim_4m', 'taille_famille_gp', 'nive…

## <a name="show hide codes"></a>Afficher/Cacher les codes sources

In [28]:
display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
    .widget-text { min-width: 60ex !important; }
</style>'''))

#Toggle Code
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();

 } else {
 $('div.input').show();

 }
 code_show = !code_show
} 
//$( document ).ready(code_toggle);//commenting code disabling by default
</script>
<form action = "javascript:code_toggle()"><input type="submit" value="Afficher/Cacher"></form>''')